[View in Colaboratory](https://colab.research.google.com/github/somanshu-singla/Minor-Project-BTech/blob/master/Minor_Project_Duplicate_Question_Detector_Keras.ipynb)

In [124]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
%%time

#path for trainig, validation and testing data
train_data_file = '/content/drive/My Drive/project/train.csv'
validation_data_file = '/content/drive/My Drive/project/validation.csv'
test_data_file = '/content/drive/My Drive/project/test.csv'

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 12.4 µs


In [126]:
%%time

import pandas as pd

#using pandas to load data from csv file

train_data = pd.read_csv(train_data_file)
train_data = train_data.iloc[:, 0:6]

columns_list = ["id", "qid1", "qid2", "ques1", "ques2", "is_duplicate"]
train_data.columns = columns_list

validation_data = pd.read_csv(validation_data_file)
validation_data = validation_data.iloc[:, 0:6]
validation_data.columns = columns_list

test_data = pd.read_csv(test_data_file)
test_data = test_data.iloc[:, 0:6]
test_data.columns = columns_list

#checking the data
print(train_data["ques1"][0:10], train_data["ques2"][0:10], train_data["is_duplicate"][0:10])

0    What is the step by step guide to invest in sh...
1    What is the story of Kohinoor (Koh-i-Noor) Dia...
2    How can I increase the speed of my internet co...
3    Why am I mentally very lonely? How can I solve...
4    Which one dissolve in water quikly sugar, salt...
5    Astrology: I am a Capricorn Sun Cap moon and c...
6                                  Should I buy tiago?
7                       How can I be a good geologist?
8                      When do you use シ instead of し?
9    Motorola (company): Can I hack my Charter Moto...
Name: ques1, dtype: object 0    What is the step by step guide to invest in sh...
1    What would happen if the Indian government sto...
2    How can Internet speed be increased by hacking...
3    Find the remainder when [math]23^{24}[/math] i...
4              Which fish would survive in salt water?
5    I'm a triple Capricorn (Sun, Moon and ascendan...
6    What keeps childern active and far from phone ...
7            What should I do to be a 

In [127]:
%%time

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
CPU times: user 48.4 ms, sys: 9.07 ms, total: 57.5 ms
Wall time: 185 ms


In [128]:
%%time

import re
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\'\"]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

#removing punctuations and stopwords(words like the,is)
def text_preprocess(text):
    text = text.lower()
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(BAD_SYMBOLS_RE, '', text)
    text = [word for word in word_tokenize(text) if word not in string.punctuation]
    #TODO: keep stopwords if removing it improves accuracy
    text = [word for word in text if word not in set(stopwords.words('english'))]
    return text

CPU times: user 0 ns, sys: 31 µs, total: 31 µs
Wall time: 40.8 µs


In [129]:
#testing text_preprocesstext
print(text_preprocess("This is Somanshu's notebook!"))

['somanshu', 'notebook']


In [130]:
%%time

train_QUES1_LIST = [text_preprocess(text) for text in train_data["ques1"]]
train_QUES2_LIST = [text_preprocess(text) for text in train_data["ques2"]]

validation_QUES1_LIST = [text_preprocess(text) for text in validation_data["ques1"]]
validation_QUES2_LIST = [text_preprocess(text)for text in validation_data["ques2"]]

test_QUES1_LIST = [text_preprocess(text) for text in test_data["ques1"]]
test_QUES2_LIST = [text_preprocess(text) for text in test_data["ques2"]]

#checking processed list
print(train_QUES1_LIST[:3])
print(train_QUES2_LIST[:3])

[['step', 'step', 'guide', 'invest', 'share', 'market', 'india'], ['story', 'kohinoor', 'kohinoor', 'diamond'], ['increase', 'speed', 'internet', 'connection', 'using', 'vpn']]
[['step', 'step', 'guide', 'invest', 'share', 'market'], ['would', 'happen', 'indian', 'government', 'stole', 'kohinoor', 'kohinoor', 'diamond', 'back'], ['internet', 'speed', 'increased', 'hacking', 'dns']]
CPU times: user 1min 35s, sys: 13 s, total: 1min 48s
Wall time: 1min 48s


In [131]:
%%time

import numpy as np

#dictionary (word --> embeddings) key = word, value = embeddings 

def load_GloVe(filename):
    embeddings_dict = {}
    print("Loading Glove Model")
    f = open(filename,'r')
    for line in f:
        word_with_embeddings = line.split()
        word = word_with_embeddings[0]
        embedding = np.array([float(val) for val in word_with_embeddings[1:]])
        embeddings_dict[word] = embedding
    print("Done.",len(embeddings_dict),"words loaded!")
    return embeddings_dict
  
 
#loading the glove file

GLOVE_FILE = '/content/drive/My Drive/project/glove.6B.300d.txt'
embeddings_dict = load_GloVe(GLOVE_FILE)

#checking word embedding
print(embeddings_dict['work'])
  

Loading Glove Model
Done. 400000 words loaded!
[ 7.1531e-02  2.9840e-01 -2.4760e-01 -2.0701e-02 -2.4787e-01 -1.2545e-01
 -4.6107e-01 -3.6251e-01 -1.4852e-01 -2.0094e+00 -2.6774e-02 -3.5909e-01
  2.0173e-01  2.3331e-01  9.6492e-02 -2.3819e-01  1.6514e-01 -5.1073e-02
 -1.7086e-01  2.4222e-01 -2.1840e-02  1.8782e-01  2.8954e-01  3.6629e-02
 -1.9667e-01  3.4200e-01  9.6643e-02 -4.6427e-02 -3.9437e-02  2.2158e-01
  7.4511e-01  7.2303e-01 -2.4423e-01  2.9758e-01 -7.5989e-01  3.2769e-01
  1.1023e-01  6.2215e-02 -1.2149e-01 -3.0053e-01 -5.4209e-02 -1.2295e-01
 -1.3716e-01 -4.0009e-01 -3.2948e-01  3.4462e-01  1.1309e-01  4.5889e-01
 -4.1368e-01 -2.6152e-01  5.2891e-01 -7.0503e-02  2.0032e-01  3.0537e-01
 -1.6813e-01  4.3551e-01 -1.6354e-01  6.1143e-02 -1.9790e-01  2.1224e-01
  1.9729e-01  1.1441e-01  3.0642e-01  2.1874e-02 -1.3517e-01 -2.6030e-01
  3.8787e-01  1.3788e-01  2.8582e-01 -2.1430e-01  5.2672e-02 -3.2403e-01
  1.5975e-01  3.2645e-01 -7.6972e-02  7.2519e-02 -2.0695e-01  4.7658e-01
  1.

In [0]:

#hyperparameters values
num_epochs      = 64
batch_size      = 64
wordvectdim     = 300
sentencepad     = 20
num_classes     = 2
cnnfiltersize   = (1,300)
poolsize        = (sentencepad,1)
num_filters     = 1800
cnninitial      = 'he_uniform'
relu            = 'relu'
dense1size      = 1800
dense2size      = 600
denseinitial    = 'he_uniform'
denseactivate   = 'tanh'
outputactivate  = 'softmax'
optimizer       = 'SGD'
num_channel     = 1
lossfunction    = 'categorical_crossentropy'

In [133]:
%%time

foo1=0
foo2=0
#prepares input for CNN model
def prepare_input(sentence_list_1,sentence_list_2):
    input_list_1,input_list_2 = np.zeros((len(sentence_list_1),num_channel,sentencepad,wordvectdim)) , np.zeros((len(sentence_list_2),num_channel,sentencepad,wordvectdim))
    i = 0
    for i in range(len(sentence_list_1)):
      sentence_matrix_1 = np.zeros((num_channel,sentencepad,wordvectdim))
      sentence_matrix_2 = np.zeros((num_channel,sentencepad,wordvectdim))
      foo = 0
      j = 1 
      for word in sentence_list_1[i]:
        if j > sentencepad:
          break
        if word in embeddings_dict:
          sentence_matrix_1[0][j-1] = embeddings_dict[word]
        else:
          sentence_matrix_1[0][j-1] = np.zeros(wordvectdim)
        j = j+1
       

      j = 1
      foo=0
      for word in sentence_list_2[i]:
        if j > sentencepad:
          break
        if word in embeddings_dict:
          sentence_matrix_2[0][j-1] = embeddings_dict[word]
        else:
          sentence_matrix_2[0][j-1] = np.zeros(wordvectdim)
        j = j+1
          
      input_list_1[i] = (sentence_matrix_1)
      input_list_2[i] = (sentence_matrix_2)
    
    return input_list_1,input_list_2          
    

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 21 µs


In [134]:
%%time

#testing the prepared input
input_train_list_1,input_train_list_2 = prepare_input(train_QUES1_LIST,train_QUES2_LIST)


input_validation_list_1,input_validation_list_2 = prepare_input(validation_QUES1_LIST,validation_QUES2_LIST)

input_test_list_1,input_test_list_2 = prepare_input(test_QUES1_LIST,test_QUES2_LIST)


print("Train List Dimension : ",input_train_list_1.shape)
print("Validate List Dimension : ",input_validation_list_1.shape)
print("Test List Dimension : ",input_test_list_1.shape)




Train List Dimension :  (20000, 1, 20, 300)
Validate List Dimension :  (2000, 1, 20, 300)
Test List Dimension :  (2000, 1, 20, 300)
CPU times: user 1.39 s, sys: 1.01 s, total: 2.4 s
Wall time: 2.4 s


In [0]:
import keras
from keras.layers import Input
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import MaxPooling2D
from keras.layers import subtract
from keras.layers import multiply
from keras.layers import maximum
from keras.layers import concatenate


def create_model():
  
    input_layer_1 = Input(shape=(num_channel,sentencepad,wordvectdim))
    input_layer_2 = Input(shape=(num_channel,sentencepad,wordvectdim))
    
    convolution_layer = Conv2D(filters=num_filters,kernel_size=cnnfiltersize,padding="valid",data_format="channels_first",activation=relu,kernel_initializer=cnninitial)
    
    max_pool_layer = MaxPooling2D(pool_size=poolsize,padding="valid",data_format="channels_first")
    
    flatten_layer = Flatten(data_format = "channels_first")
    
    conv_layer_output_1 = convolution_layer(input_layer_1)
    conv_layer_output_2 = convolution_layer(input_layer_2)
    
    pool_layer_output_1 = max_pool_layer(conv_layer_output_1)
    pool_layer_output_2 = max_pool_layer(conv_layer_output_2)
    
    flatten_layer_output_1 = flatten_layer(pool_layer_output_1)
    flatten_layer_output_2 = flatten_layer(pool_layer_output_2)
    
    sub_1_1 = subtract(inputs=[flatten_layer_output_1,flatten_layer_output_2])
    sub_1_2 = subtract(inputs=[flatten_layer_output_2,flatten_layer_output_1])
    
    abs_layer = maximum(inputs = [sub_1_1,sub_1_2])
    mul_layer = multiply(inputs = [flatten_layer_output_1,flatten_layer_output_2])
    
    concatenate_layer = concatenate(inputs=[abs_layer,mul_layer])
    
    dense_layer_1 = Dense(units=dense1size,activation=denseactivate,kernel_initializer=denseinitial)(concatenate_layer)
    
    dense_layer_2 = Dense(units=dense2size,activation=denseactivate,kernel_initializer=denseinitial)(dense_layer_1)
    
    output_layer = Dense(units=num_classes,activation=outputactivate)(dense_layer_2)
    
    model = Model(inputs=[input_layer_1,input_layer_2],outputs=output_layer)
    
    print(model.summary())
    
    return model
    
  

In [138]:
model = create_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 1, 20, 300)   0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 1, 20, 300)   0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 1800, 20, 1)  541800      input_21[0][0]                   
                                                                 input_22[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_11 (MaxPooling2D) (None, 1800, 1, 1)   0           conv2d_11[0][0]                  
          

In [0]:
from keras.losses import categorical_crossentropy,sparse_categorical_crossentropy
from keras.optimizers import SGD
optimizer = SGD(lr=0.01)
model.compile(loss = "categorical_crossentropy", optimizer = optimizer,metrics=['accuracy'])

In [140]:
from keras.utils import to_categorical
model.fit([input_train_list_1, input_train_list_2], to_categorical(train_data['is_duplicate']), batch_size = batch_size , epochs= 6 ,verbose=1,validation_data = ([input_validation_list_1,input_validation_list_2],to_categorical(validation_data['is_duplicate'])))

Train on 20000 samples, validate on 2000 samples
Epoch 1/6
20000/20000 [==============================] - 15s 759us/step - loss: 0.5661 - acc: 0.7023 - val_loss: 0.5028 - val_acc: 0.7600
Epoch 2/6
20000/20000 [==============================] - 14s 710us/step - loss: 0.4885 - acc: 0.7500 - val_loss: 0.4802 - val_acc: 0.7585
Epoch 3/6
20000/20000 [==============================] - 14s 721us/step - loss: 0.4616 - acc: 0.7694 - val_loss: 0.5070 - val_acc: 0.7275
Epoch 4/6
20000/20000 [==============================] - 15s 726us/step - loss: 0.4421 - acc: 0.7826 - val_loss: 0.6132 - val_acc: 0.6910
Epoch 5/6
20000/20000 [==============================] - 15s 729us/step - loss: 0.4268 - acc: 0.7923 - val_loss: 0.5132 - val_acc: 0.7325
Epoch 6/6
20000/20000 [==============================] - 14s 724us/step - loss: 0.4142 - acc: 0.7996 - val_loss: 0.4781 - val_acc: 0.7520


In [141]:
 #predicting socres
scores = model.predict([input_test_list_1,input_test_list_2],batch_size=batch_size,verbose = 1)

2000/2000 [==============================] - 1s 370us/step


In [142]:
print(scores)
ss=[]
for i in range(len(scores)):
    ss.append(np.argmax(scores[i]))
scores1=ss


[[0.9406146  0.05938545]
 [0.27021953 0.72978044]
 [0.9069184  0.09308164]
 ...
 [0.99686867 0.00313129]
 [0.29620543 0.7037946 ]
 [0.5072218  0.49277815]]


In [143]:
print(scores1)

[0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 

In [144]:
#number of wrong predictions
wrong = 0
for i in range(len(scores1)):
    if scores1[i] != test_data['is_duplicate'][i]:
      wrong = wrong + 1
      
print("Accuracy : ",1-wrong/len(scores1))

Accuracy :  0.7575000000000001


In [92]:
!-smi

/bin/bash: -c: option requires an argument
